In [1]:
# default_exp write_data 

# Write data to docanno
> write data for annotation.

In [2]:
# export
%load_ext autoreload
%autoreload 2
import pandas as pd
from tqdm import tqdm_notebook
from whatsapp_analytics import translation,preprocess_lib
import pickle
from os import path
import parmap
import numpy as np
from langdetect import DetectorFactory
DetectorFactory.seed = 0
from langdetect import detect
import re
import uuid
from account import baseurl,name,password
import json
import random
parent_path='../Data/New_Data_15-06-2020/'
import math

In [3]:
# hide
if path.exists(parent_path+'round_info.pkl'):
    with open(parent_path+'round_info.pkl', 'rb') as f:
        round_info = pickle.load(f)
else:
    print("no_round_info")

In [4]:
round_info

11

In [5]:
# hide
annotation_data='../Data/Annotations_to_send/'
if path.exists(annotation_data+'annotation_'+str(round_info)+'.pkl'):
    sample_df = pd.read_pickle(annotation_data+'annotation_'+str(round_info)+'.pkl')

In [6]:
# export
def wrong_translation(df):
    wrong_trans_list=[]
    for index,row in tqdm_notebook(df.iterrows(),total=len(df)):
        try:
            lang=detect(row['translated'])
        except:
            lang='unk'
        if(lang!='en'):
            wrong_trans_list.append(1)
            continue
        org_tokens=preprocess_lib.preprocess_sent(row['message_text'],params={'remove_numbers': False,'remove_emoji': False, 'remove_stop_words': False, 'tokenize': True})
        trans_tokens=preprocess_lib.preprocess_sent(row['translated'],params={'remove_numbers': False,'remove_emoji': False, 'remove_stop_words': False, 'tokenize': True})
        if((abs(len(org_tokens)-len(trans_tokens))/len(org_tokens))>0.2):
                wrong_trans_list.append(1)
        else:
                wrong_trans_list.append(0)
    df['wrong_translation']=wrong_trans_list
    return df

In [7]:
sample_df=wrong_translation(sample_df)

In [8]:
# hide
from doccano_api_client import DoccanoClient

In [9]:
# hide
doccano_client = DoccanoClient(
    baseurl,name,password
)

# get basic information about the authorized user
r_me = doccano_client.get_me()

# print the details from the above query
print(r_me.json())


<Response [200]>
{'id': 1, 'username': 'Punyajoy', 'first_name': 'Punyajoy', 'last_name': 'Saha', 'email': 'punyajoysaha1998@gmail.com', 'is_superuser': True}


In [10]:
# hide
import json
projects=doccano_client.get_project_list().json()

In [11]:
# hide
with open('user_dict.json','r') as f:
    user_dict_keyname = json.load(f)

In [12]:
# hide
user_dict_keyname


for key in user_dict_keyname:
    user_dict_keyname[key]['count']=0

In [13]:
# hide
user_dict_keyname

{'Divyanshu': {'id': 41, 'count': 0},
 'Debayu': {'id': 44, 'count': 0},
 'Danish': {'id': 54, 'count': 0},
 'Sharat': {'id': 53, 'count': 0},
 'Kukkadapu': {'id': 50, 'count': 0},
 'Adarsh': {'id': 52, 'count': 0},
 'Yuvraj': {'id': 49, 'count': 0},
 'Vineeth': {'id': 51, 'count': 0},
 'Samanway': {'id': 45, 'count': 0}}

In [14]:
# hide
users_to_ignore = ['Danish','Samanway','Adarsh','Sharat']
num_users=len(user_dict_keyname.keys())-len(users_to_ignore)
max_per_round=math.floor(len(sample_df)*3/num_users)


In [15]:
max_per_round

120

In [20]:
# export
def prepare_data(actual_text,translated):
    
    text ='::Translated_Part:: \n\n'
    try:
        text+=translated+' \n\n '
    except:
        text+="!! Translation not available !! sorry for the trouble"+ '\n\n'
    text+='::Actual Part:: \n\n'
    text+=actual_text +' \n\n '
    return text
    

def distribute(user_dict,sample_df,max_per_round):
    dict1_user={}
    for index,row in tqdm_notebook(sample_df.iterrows(),total=len(sample_df)):
        text=prepare_data(row['message_text'],row['translated'])
        user_tuples=sorted(user_dict.items(), key=lambda item: (item[1]['count'],random.random()))
        unique_ids=row['unique_ids']
        count_user=0
        i=0
        one_round=0
        already_selected=[]
        while(count_user<=2):
            if(i>=len(user_dict_keyname.keys())):
                i=0
                one_round+=1
            if(one_round>50):
                return
            user=user_tuples[i][0]
            i=i+1
            ### considering users 
            ## ignore users not annotating
            if user in already_selected:
                continue
            if user in users_to_ignore:
                continue
            ### ignore Vineeth for wrong translation
            if user=='Vineeth' and row['wrong_translation']==1:
                continue
            ## ignore users if he crosses max this round
            if(one_round<=3):
                try:
                    if(len(dict1_user[user])>max_per_round):
                        continue
                except KeyError:
                    pass

            temp=text+' $$'+unique_ids[count_user]+'$$'
            try:
                dict1_user[user].append(temp)
            except KeyError:
                dict1_user[user] = [temp]
            user_dict[user]['count']+=1
            count_user+=1
            already_selected.append(user)
            
        
    return dict1_user,user_dict




def write_to_all(user_dict,sample_df):
    dict1_user={}
    for user in user_dict:
        if user in users_to_ignore:
                continue
        for index,row in tqdm_notebook(sample_df.iterrows(),total=len(sample_df)):
            text=prepare_data(row['message_text'],row['translated'])
            user_tuples=sorted(user_dict.items(), key=lambda item: (item[1]['count'],random.random()))
            unique_ids=row['unique_ids']
            temp=text+' $$'+unique_ids[0]+'$$'
            try:
                dict1_user[user].append(temp)
            except KeyError:
                dict1_user[user] = [temp]
    return dict1_user

In [21]:
# hide
user_text_map,user_dict=distribute(user_dict_keyname.copy(),sample_df,max_per_round)

In [22]:
user_text_map=write_to_all(user_dict_keyname.copy(),sample_df)

In [23]:
user_text_map.keys()

dict_keys(['Divyanshu', 'Debayu', 'Kukkadapu', 'Yuvraj', 'Vineeth'])

In [24]:
# export
def upload_to_json(user_text_map,projects):
    for user in tqdm_notebook(user_text_map.keys(),total=len(user_text_map)):
        temp=user_text_map[user]
        json_list=[]
        for item in temp:
            json_list.append({'text':item})
        with open('output.json', 'w') as fout:
            for item in json_list:
                fout.write(json.dumps(item))
                fout.write('\n')
        for project in projects:
            project_name='Annotation_'+user
            if(project_name == project['name']):
                project_id=project['id']
                break
        doccano_client.post_doc_upload(project_id,'json','output.json')

In [25]:
# hide
upload_to_json(user_text_map,projects)

In [20]:
# hide
json_object = json.dumps(user_dict, indent = 4) 
with open('user_dict.json','w') as outfile:
    outfile.write(json_object) 

In [21]:
# hide
user_dict

{'Divyanshu': {'id': 41, 'count': 300},
 'Debayu': {'id': 44, 'count': 0},
 'Danish': {'id': 54, 'count': 0},
 'Sharat': {'id': 53, 'count': 300},
 'Kukkadapu': {'id': 50, 'count': 301},
 'Adarsh': {'id': 52, 'count': 0},
 'Yuvraj': {'id': 49, 'count': 300},
 'Vineeth': {'id': 51, 'count': 299},
 'Samanway': {'id': 45, 'count': 0}}